In [6]:
import pandas as pd

In [7]:
file = 'C:\\Users\\daun5\\jupyter_notebook\\EC-Advance_daun\\room_list_Gongneung-dong.csv'

In [8]:
room_data = pd.read_csv(file, encoding='cp949')

In [9]:
def split(x): #price_info 전처리
    x = x.replace('[', ' ').replace("'", ' ').replace(']', ' ')
    x = x.split(',')
    
    return x

In [10]:
i = 0; #관리비 전처리
for data in room_data['maintenance_cost_str']:
    if data == '없음':
        room_data['maintenance_cost_str'][i] = 0
    else:
        room_data['maintenance_cost_str'][i] = float(data[:-3])
    i = i + 1;

<ipython-input-10-ea54dfc5b833>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_data['maintenance_cost_str'][i] = float(data[:-3])
<ipython-input-10-ea54dfc5b833>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_data['maintenance_cost_str'][i] = 0


In [11]:
room_data['monthly_rent'] = room_data['price_info_str'].apply(lambda x:split(x)).str[1]
room_data['deposit'] = room_data['price_info_str'].apply(lambda x:split(x)).str[0]
#price_info 를 월세와 보증금으로 분리

In [12]:
room_data.insert(6, 'monthly_total', 0.0)
#월세+관리비 총합으로 monthly_total 열 만듦

In [13]:
room_data['maintenance_cost_str'] = pd.to_numeric(room_data['maintenance_cost_str'])
room_data['monthly_rent'] = pd.to_numeric(room_data['monthly_rent'])
i = 0;
for j in room_data['deposit']:
    if('억' in j):
        room_data['deposit'][i] = room_data['deposit'][i].replace('억', ' ')
        room_data['deposit'][i] = room_data['deposit'][i].replace(' ', '')
    i += 1
room_data['deposit'] = pd.to_numeric(room_data['deposit'])
#관리비, 월세, 보증금 문자형을 숫자형으로 변환

In [14]:
i = 0;
for j in room_data['room_id']:
    room_data['monthly_total'][i] = room_data['monthly_rent'][i] + room_data['maintenance_cost_str'][i]
    i += 1
#보증금+월세해서 monthly_total에 대입

<ipython-input-14-2c63176b6ed5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_data['monthly_total'][i] = room_data['monthly_rent'][i] + room_data['maintenance_cost_str'][i]


In [15]:
room_data = room_data.drop(columns=['price_info_str', 'maintenance_cost_str', 'monthly_rent'], axis=1)

In [16]:
j = 0 
for i in room_data['room_floor_str']: #층수 전처리 중층은 삭제, 반지층은 -1로
    if (i == '중층'):
        room_data = room_data.drop(index = [j], axis = 0)
    elif (i == '반지층'):
        room_data['room_floor_str'][j] = -1
    else:
        room_data['room_floor_str'][j] = i.replace('층', ' ')
    j += 1
room_data['room_floor_str'] = pd.to_numeric(room_data['room_floor_str'])

<ipython-input-16-1929a96226b7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_data['room_floor_str'][j] = i.replace('층', ' ')
<ipython-input-16-1929a96226b7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_data['room_floor_str'][j] = -1


In [18]:
room_data

,room_id,room_size,room_floor_str,room_options,monthly_total,direction_str,building_approval_date_str,deposit
0,61318646c5d09b5e1b68fdda,19.83,2,"['신발장', '인덕션', '전자도어락']",40.0,동,2004.02.26,500
1,6130637514ec260c50318df4,19.83,3,"['옷장', '신발장', '인덕션', '전자레인지']",54.0,남서,2012.10.24,100
2,6131e5ca24c0635678e22e80,17.27,10,"['신발장', '인덕션', '전자도어락']",66.0,북,2016.02.05,1000
3,61163cb6a5d8fd259e38b6c1,33.05,4,"['책상', '옷장', '신발장', '가스레인지', '전자레인지', '전자도어락']",54.0,남,2012.07.31,1000
4,612c5be69b1c061b93396d0d,26.44,1,['가스레인지'],25.0,동,1975.11.29,2000
...,...,...,...,...,...,...,...,...
199,613703a6d2991e74b02b741f,16.52,-1,"['책상', '옷장', '신발장', '인덕션', '전자레인지', '전자도어락']",33.0,NaN,-,200
200,6049cd82db47993cca255b69,16.52,2,"['책상', '옷장', '신발장', '가스레인지', '인덕션', '전자레인지', '...",35.0,NaN,-,300
201,613592cfd9d61b4af84dccb5,17.00,6,"['옷장', '신발장', '가스레인지', '전자도어락']",40.0,NaN,-,500
202,566537acf3242d794152a719,23.10,3,"['책상', '옷장', '신발장', '인덕션', '전자도어락', '전자레인지', '...",50.0,NaN,-,500
